In [1]:
from xml.dom import minidom
import re
import pandas as pd
import sys
import pickle
import numpy as np
import time
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
#["game","gaming","gamed","games"]

ps = PorterStemmer()
sys.__stdout__=sys.stdout
#TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    #text=re.sub('\n',' ',text)
    #text=re.sub('-',' ',text)
    #text=re.sub('[\'"]','',text)
    #text=re.sub('[()?.@*#&!,$]','',text)
    text=re.sub('[^a-z0-9]',' ',text)
    return text#TAG_RE.sub('', text)

def make_dictionary(topics,max_doc,root):
    index=0
    wordmap={}
    word_count=[]
    for topic in topics:
        print(topic)
        mydoc = minidom.parse(root+topic+'.xml')
        docs = mydoc.getElementsByTagName('row')
        doc_count=0
        for doc in docs:
            if doc_count>=max_doc:
                break
            doc=doc.attributes['Body'].value
            doc=doc.lower()
            doc = remove_tags(doc)
            words=doc.split(' ')
            for word in words:
                word=ps.stem(word)
                if word in s_words:
                    continue
                if word not in wordmap:
                    wordmap[word]=index
                    word_count.append(int(1))
                    index=index+1
                elif word in wordmap:
                    ind=wordmap[word]
                    word_count[ind]=int(word_count[ind])+1
            doc_count=doc_count+1
    return wordmap,word_count
def vectorize_seq(topics,max_doc,wordmap,root):
    n_row=len(topics)*(max_doc)
    vectors=np.zeros((n_row,len(wordmap)),dtype=np.float64)
    vct_label=[]
    row=0
    for topic in topics:
        print(topic)
        mydoc = minidom.parse(root+topic+'.xml')
        docs = mydoc.getElementsByTagName('row')
        doc_count=0
        for doc in docs:
            if doc_count>=max_doc or doc_count>=len(docs):
                break
            #vector = [0]*(len(wordmap)+2)
            doc=doc.attributes['Body'].value
            doc=doc.lower()
            doc = remove_tags(doc)
            words=doc.split(' ')
                     
            if len(words)<5:
                continue
            for word in words:
                word=ps.stem(word)
                if word in wordmap:
                    ind=wordmap[word]
                    vectors[row,ind]=vectors[row,ind]+1
            row=row+1
            vct_label.append(topic)
            word_list.append(words)
            doc_count=doc_count+1
    return vectors,vct_label


In [2]:
root='Dataset/Training/'
root_test='Dataset/Test/'
f=open('Stopwords.txt')
s_words=f.read()
s_words=s_words.split('\n')
s_words.append('')

topics=['3d_Printer','Coffee','Arduino','Astronomy','Biology']#,'Chess','Cooking','Law','Space','Windows_Phone','Wood_Working']
#topics=['Anime']


 
#wordmap=make_dictionary(topics,100,root)
#vectors=vectorize_seq(topics,100,wordmap,root)
word_list=[]
write_backup=1
max_docs_train=500
if write_backup==1:
    wordmap,wc=make_dictionary(topics,max_docs_train,root)
    vectors,vct_labels=vectorize_seq(topics,max_docs_train,wordmap,root)
    pickle_out = open("wordmap_test.pickle","wb")
    pickle.dump(wordmap, pickle_out)
    pickle_out.close()
    pickle_out = open("vectors_test.pickle","wb")
    pickle.dump(vectors, pickle_out)
    pickle_out.close()

read_backup=0
if read_backup==1:
    print('Reading backup')
    pickle_read = open("wordmap_500.pickle","rb")
    wordmap=pickle.load(pickle_read)
    pickle_read.close()

    pickle_read = open("vectors_500.pickle","rb")
    vectors=pickle.load(pickle_read)
    pickle_read.close()

#words=list(wordmap.keys())
#freq=list(wordmap.values())
#word_df=pd.DataFrame({'words':words,'freq':freq})
#word_df=word_df.sort_values('freq',ascending=False)
    



3d_Printer
Coffee
Arduino
Astronomy
Biology
3d_Printer
Coffee
Arduino
Astronomy
Biology


In [3]:
print(len(wordmap))
#  no stem 11625
# stem 8392

14862


In [4]:
#print(word_list)

In [5]:
words=list(wordmap.keys())
freq=list(wordmap.values())
word_df=pd.DataFrame(words,index=list(wordmap.values()))
word_df=word_df.sort_index()
word_df['freq']=wc
word_df=word_df.sort_values('freq',ascending=False)
print(word_df[0:50])
#print(len(vectors))

              0  freq
220         thi  2624
9           use  2152
4002      coffe  1773
2000       code  1694
162           s  1624
1848    arduino  1190
65            t  1174
1         print   929
92     question   800
44         time   784
262          ha   694
38          ani   642
79       differ   638
602   blockquot   628
408       water   522
50         onli   504
143        work   496
658          wa   487
383         way   483
2113        amp   482
5171       star   472
14      printer   461
2305        pin   458
68         veri   449
4228       brew   436
2270      board   433
2250        pre   425
125      filter   394
15            m   393
177         doe   388
86     filament   379
887        cell   367
159         don   365
955       stack   360
3573     serial   356
56       becaus   349
212     process   348
4224       bean   346
174        look   345
300       gener   345
4284    caffein   344
9635     planet   343
956       imgur   335
295        good   332
360      e

In [6]:
def eucl_dis(test_vect,vectors,vct_labels):
    h_dists=[]
    for row in range(len(vct_labels)):
        vect=vectors[row]
        dist=np.abs(vect-test_vect)
        dist=np.square(dist)
        dist=np.sum(dist)
        dist=np.sqrt(dist)
        label=vct_labels[row]
        h_dists.append([dist,label])
    return h_dists
def humm_dis(test_vector,vectors,vct_labels):
    h_dists=[]
    
    test_vect=np.array(test_vector>0,dtype=np.uint8)
    vects=np.array(vectors>0,dtype=np.uint8)
    #test_vect=test_vector
    #vects=vectors
    for row in range(len(vct_labels)):
        vect=vects[row]
        label=vct_labels[row]
        dist=np.abs(vect-test_vect)
        #dist=dist*[test_vect!=0]
        dist=np.sum(dist)
        h_dists.append([dist,label])
    return h_dists
def cos_dis(test_vect,vectors,vct_labels):
    h_dists=[]
    
    vectors_bin= np.array(vectors>0,dtype=np.uint8)
    wc_vect=np.sum(vectors_bin,axis=0)
    no_docs=len(vectors)
    
    IDF=no_docs/wc_vect
    IDF=np.log(IDF)
    
    for row in range(len(vct_labels)):
        train_vect=vectors[row]
        
        train_label=vct_labels[row]
        
        train_total=np.sum(train_vect)
        test_total=np.sum(test_vect)
        
        TF_test=test_vect/test_total
        test_dis=TF_test*IDF
        
        temp=np.square(test_dis)
        temp=np.sum(temp)
        test_dis_mag=np.sqrt(temp)
        
        TF_train=train_vect/train_total
        train_dis=TF_train*IDF
        
        temp=np.square(train_dis)
        temp=np.sum(temp)
        train_dis_mag=np.sqrt(temp)
        
        X=np.sum(test_dis*train_dis)
        Y=train_dis_mag*test_dis_mag
        
        dist=X/Y
        h_dists.append([dist,train_label])
    return h_dists

In [11]:
def knn(nn,test_vect,vectors,vct_labels,method):
    print('getting h dist')
    
    if method=='euc':
        h_dists=eucl_dis(test_vect,vectors,vct_labels)
    if method=='hum':
        h_dists=humm_dis(test_vect,vectors,vct_labels)
    if method=='cos':
        h_dists=cos_dis(test_vect,vectors,vct_labels)
    h_df=pd.DataFrame(h_dists,columns=['dist','label'])
    
    if method=='euc' or method=='hum':
        h_df=h_df.sort_values('dist',ascending=True)
    if method=='cos':
        h_df=h_df.sort_values('dist',ascending=False)
    
    lb=h_df['label']
    lb=lb[:nn]
    lb=lb.mode()
    lb=lb.values[0]
    return(lb)
max_docs_test=60
test_vector,true_lbs=vectorize_seq(topics,max_docs_test,wordmap,root_test)

n=len(topics)*max_docs_test
methods=['cos','euc','hum']
k_val=[1,3,5]
log=True
if log==True:
    file=open('acccuracy.txt','a')
    file.write('steming:'+'yes\n'+'wordmap :'+str(len(wordmap))+'\n')
    file.write('topics : ')
    file.write(str(topics)+'\n')
    file.write('train_docs :\n')
    no_topics=len(topics)
    tot_doc=no_topics*max_docs_train
    file.write(str(max_docs_train)+'*'+str(no_topics)+'='+str(tot_doc)+'\n')
    tot_doc_test=no_topics*max_docs_test
    file.write('test_docs :\n')
    file.write(str(max_docs_train)+'*'+str(no_topics)+'='+str(tot_doc)+'\n')
    file.close()
acc_log=''
acc_log+='******************************************************************\n'
acc_log+=str(topics)+'\n'
acc_log+=str(['no_topic','train doc per topic','train_doc','test doc per topic','test_doc','word map'])+'\n'
acc_log+=str([no_topics,max_docs_train,tot_doc,max_docs_test,tot_doc_test,str(len(wordmap))])+'\n'
acc_log+=str(['k','method','accuracy','time'])+'\n'
acc_log+='******************************************************************\n'
for k in k_val:
    for method in methods:
        acc=0
        T=time.time()
        for i in range(n):
            true_lb=true_lbs[i]
            test_vect=test_vector[i,:]
            pred_lb=knn(k,test_vect,vectors,vct_labels,method)
            print(pred_lb,true_lb)
            if pred_lb==true_lb:
                acc=acc+1
                print(pred_lb,true_lb,acc,i)
        accuracy=acc/(n)
        T=time.time()-T
        print('accuray',accuracy)
        print('Time :',T)
        if log==True:
            file=open('acccuracy.txt','a')
            file.write('accuracy '+'method :'+method+' k :'+str(k)+':')
            file.write(str(accuracy)+'\n')
            file.close()
            acc_log+=(str([k,method,accuracy,T])+'\n')
acc_log=re.sub('[\[\] ]','',acc_log)
file=open('rec.csv','a')
file.write(acc_log)
file.close()
#68.75% eucl 60 82.92
#53.33 humm 60  81.25
#87.08 cos 94.17

3d_Printer
Coffee
Arduino
Astronomy
Biology
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 1 0
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 2 1
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 3 2
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 4 3
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 5 4
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 6 5
getting h dist
Biology 3d_Printer
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 7 7
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 8 8
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 9 9
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 10 10
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 11 11
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 12 12
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 13 13
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 14 14
getting h dist
3d_Printer 3d_P

Arduino Arduino
Arduino Arduino 141 146
getting h dist
Arduino Arduino
Arduino Arduino 142 147
getting h dist
3d_Printer Arduino
getting h dist
Arduino Arduino
Arduino Arduino 143 149
getting h dist
3d_Printer Arduino
getting h dist
Arduino Arduino
Arduino Arduino 144 151
getting h dist
Arduino Arduino
Arduino Arduino 145 152
getting h dist
Arduino Arduino
Arduino Arduino 146 153
getting h dist
Arduino Arduino
Arduino Arduino 147 154
getting h dist
Arduino Arduino
Arduino Arduino 148 155
getting h dist
Arduino Arduino
Arduino Arduino 149 156
getting h dist
Arduino Arduino
Arduino Arduino 150 157
getting h dist
Arduino Arduino
Arduino Arduino 151 158
getting h dist
Arduino Arduino
Arduino Arduino 152 159
getting h dist
Arduino Arduino
Arduino Arduino 153 160
getting h dist
3d_Printer Arduino
getting h dist
Arduino Arduino
Arduino Arduino 154 162
getting h dist
Arduino Arduino
Arduino Arduino 155 163
getting h dist
Arduino Arduino
Arduino Arduino 156 164
getting h dist
Arduino Arduino
Ar

3d_Printer 3d_Printer
3d_Printer 3d_Printer 2 1
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 3 2
getting h dist
Coffee 3d_Printer
getting h dist
Astronomy 3d_Printer
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 4 5
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 5 6
getting h dist
Biology 3d_Printer
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 6 8
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 7 9
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 8 10
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 9 11
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 10 12
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 11 13
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 12 14
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 13 15
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 14 16
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 15 17
getting h dist
A

Arduino Arduino
Arduino Arduino 132 158
getting h dist
Arduino Arduino
Arduino Arduino 133 159
getting h dist
Arduino Arduino
Arduino Arduino 134 160
getting h dist
3d_Printer Arduino
getting h dist
Arduino Arduino
Arduino Arduino 135 162
getting h dist
Arduino Arduino
Arduino Arduino 136 163
getting h dist
Arduino Arduino
Arduino Arduino 137 164
getting h dist
3d_Printer Arduino
getting h dist
Arduino Arduino
Arduino Arduino 138 166
getting h dist
Arduino Arduino
Arduino Arduino 139 167
getting h dist
Arduino Arduino
Arduino Arduino 140 168
getting h dist
Arduino Arduino
Arduino Arduino 141 169
getting h dist
Arduino Arduino
Arduino Arduino 142 170
getting h dist
Biology Arduino
getting h dist
Arduino Arduino
Arduino Arduino 143 172
getting h dist
Arduino Arduino
Arduino Arduino 144 173
getting h dist
Arduino Arduino
Arduino Arduino 145 174
getting h dist
3d_Printer Arduino
getting h dist
3d_Printer Arduino
getting h dist
Astronomy Arduino
getting h dist
Arduino Arduino
Arduino Arduin

Astronomy 3d_Printer
getting h dist
Astronomy 3d_Printer
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 15 22
getting h dist
Astronomy 3d_Printer
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 16 24
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 17 25
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 18 26
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 19 27
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 20 28
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 21 29
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 22 30
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 23 31
getting h dist
Astronomy 3d_Printer
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 24 33
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 25 34
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printer 26 35
getting h dist
Arduino 3d_Printer
getting h dist
3d_Printer 3d_Printer
3d_Printer 3d_Printe

...

Astronomy Astronomy
Astronomy Astronomy 202 213
getting h dist
Astronomy Astronomy
Astronomy Astronomy 203 214
getting h dist
Astronomy Astronomy
Astronomy Astronomy 204 215
getting h dist
Astronomy Astronomy
Astronomy Astronomy 205 216
getting h dist
Astronomy Astronomy
Astronomy Astronomy 206 217
getting h dist
Astronomy Astronomy
Astronomy Astronomy 207 218
getting h dist
Astronomy Astronomy
Astronomy Astronomy 208 219
getting h dist
3d_Printer Astronomy
getting h dist
Astronomy Astronomy
Astronomy Astronomy 209 221
getting h dist
Astronomy Astronomy
Astronomy Astronomy 210 222
getting h dist
Astronomy Astronomy
Astronomy Astronomy 211 223
getting h dist
Astronomy Astronomy
Astronomy Astronomy 212 224
getting h dist
Astronomy Astronomy
Astronomy Astronomy 213 225
getting h dist
Astronomy Astronomy
Astronomy Astronomy 214 226
getting h dist
Astronomy Astronomy
Astronomy Astronomy 215 227
getting h dist
Astronomy Astronomy
Astronomy Astronomy 216 228
getting h dist
Astronomy Astronomy

Coffee Coffee
Coffee Coffee 53 62
getting h dist
3d_Printer Coffee
getting h dist
Coffee Coffee
Coffee Coffee 54 64
getting h dist
Coffee Coffee
Coffee Coffee 55 65
getting h dist
Biology Coffee
getting h dist
Biology Coffee
getting h dist
Coffee Coffee
Coffee Coffee 56 68
getting h dist
Coffee Coffee
Coffee Coffee 57 69
getting h dist
Coffee Coffee
Coffee Coffee 58 70
getting h dist
Coffee Coffee
Coffee Coffee 59 71
getting h dist
Coffee Coffee
Coffee Coffee 60 72
getting h dist
Coffee Coffee
Coffee Coffee 61 73
getting h dist
3d_Printer Coffee
getting h dist
Coffee Coffee
Coffee Coffee 62 75
getting h dist
Coffee Coffee
Coffee Coffee 63 76
getting h dist
Biology Coffee
getting h dist
Coffee Coffee
Coffee Coffee 64 78
getting h dist
Coffee Coffee
Coffee Coffee 65 79
getting h dist
Coffee Coffee
Coffee Coffee 66 80
getting h dist
Coffee Coffee
Coffee Coffee 67 81
getting h dist
Coffee Coffee
Coffee Coffee 68 82
getting h dist
Coffee Coffee
Coffee Coffee 69 83
getting h dist
Coffee Coff

Astronomy Astronomy
Astronomy Astronomy 181 225
getting h dist
Astronomy Astronomy
Astronomy Astronomy 182 226
getting h dist
Biology Astronomy
getting h dist
3d_Printer Astronomy
getting h dist
Astronomy Astronomy
Astronomy Astronomy 183 229
getting h dist
Astronomy Astronomy
Astronomy Astronomy 184 230
getting h dist
Astronomy Astronomy
Astronomy Astronomy 185 231
getting h dist
Astronomy Astronomy
Astronomy Astronomy 186 232
getting h dist
Astronomy Astronomy
Astronomy Astronomy 187 233
getting h dist
Astronomy Astronomy
Astronomy Astronomy 188 234
getting h dist
Astronomy Astronomy
Astronomy Astronomy 189 235
getting h dist
Astronomy Astronomy
Astronomy Astronomy 190 236
getting h dist
Astronomy Astronomy
Astronomy Astronomy 191 237
getting h dist
Astronomy Astronomy
Astronomy Astronomy 192 238
getting h dist
Biology Astronomy
getting h dist
Astronomy Biology
getting h dist
Biology Biology
Biology Biology 193 241
getting h dist
Coffee Biology
getting h dist
Biology Biology
Biology 

Coffee Coffee
Coffee Coffee 72 78
getting h dist
Coffee Coffee
Coffee Coffee 73 79
getting h dist
Coffee Coffee
Coffee Coffee 74 80
getting h dist
Coffee Coffee
Coffee Coffee 75 81
getting h dist
Coffee Coffee
Coffee Coffee 76 82
getting h dist
Coffee Coffee
Coffee Coffee 77 83
getting h dist
Coffee Coffee
Coffee Coffee 78 84
getting h dist
Coffee Coffee
Coffee Coffee 79 85
getting h dist
Coffee Coffee
Coffee Coffee 80 86
getting h dist
Astronomy Coffee
getting h dist
Coffee Coffee
Coffee Coffee 81 88
getting h dist
Coffee Coffee
Coffee Coffee 82 89
getting h dist
Coffee Coffee
Coffee Coffee 83 90
getting h dist
Coffee Coffee
Coffee Coffee 84 91
getting h dist
Coffee Coffee
Coffee Coffee 85 92
getting h dist
Coffee Coffee
Coffee Coffee 86 93
getting h dist
Arduino Coffee
getting h dist
Coffee Coffee
Coffee Coffee 87 95
getting h dist
Coffee Coffee
Coffee Coffee 88 96
getting h dist
Coffee Coffee
Coffee Coffee 89 97
getting h dist
Coffee Coffee
Coffee Coffee 90 98
getting h dist
Coffee 

Astronomy Astronomy
Astronomy Astronomy 209 231
getting h dist
Astronomy Astronomy
Astronomy Astronomy 210 232
getting h dist
Astronomy Astronomy
Astronomy Astronomy 211 233
getting h dist
Astronomy Astronomy
Astronomy Astronomy 212 234
getting h dist
Astronomy Astronomy
Astronomy Astronomy 213 235
getting h dist
Astronomy Astronomy
Astronomy Astronomy 214 236
getting h dist
Arduino Astronomy
getting h dist
Astronomy Astronomy
Astronomy Astronomy 215 238
getting h dist
Astronomy Astronomy
Astronomy Astronomy 216 239
getting h dist
Astronomy Biology
getting h dist
3d_Printer Biology
getting h dist
Biology Biology
Biology Biology 217 242
getting h dist
3d_Printer Biology
getting h dist
Coffee Biology
getting h dist
Astronomy Biology
getting h dist
Biology Biology
Biology Biology 218 246
getting h dist
Biology Biology
Biology Biology 219 247
getting h dist
Coffee Biology
getting h dist
Astronomy Biology
getting h dist
Astronomy Biology
getting h dist
Biology Biology
Biology Biology 220 25

In [10]:
print(len(vectors))
print(len(vct_labels))
vects=np.array(vectors>0,dtype=np.uint8)
print(len(vects))

2500
2500
2500


In [ ]:
from matplotlib import pyplot as plt
vct=[vectors[12][:500]]
test=vectors[15][:498]
dis=eucl_dis(test,vct)
print(vct)
print(test)
print(dis[0][0])
x=range(0,500)
xt=range(0,498)
plt.plot(x,vct[0],xt,test)
plt.show

In [ ]:
def eucl_dis(test_vector,vectors):
    h_dists=[]
    for vector in vectors:
        vect=np.array(vector[:-2])
        vect[vect>1]=1
        label=vector[-1]
        test_vect=np.array(test_vector)
        dist=np.abs(vect-test_vect)
        #dist=np.sum(dist)
        h_dists.append([dist,label])
    return h_dists
test_vector=vectors[0][:-2]
vecto=[vectors[0],vectors[1]]
h_dists=cos_dis(test_vector,vecto)
#h_df=pd.DataFrame(h_dists)
#h_df=h_df.sort_values(0)
print(h_dists)
#print((vectors[15]))

In [ ]:
for i in range(len(vectors)):
    vct=vectors[i][:-2]
    if np.sum(vct)==0:
        break
print(i,word_list[185])

In [109]:
raw='I am hasib\n,i "am" a student() so? @i do& my-duties! porp. and' 
#text=re.sub('\n',' ',raw)
text=raw.lower()
text=re.sub('[^a-z0-9]',' ',text)
#text=re.sub('-',' ',text)
#text=re.sub('[\'"]','',text)
#text=re.sub('[()?.@*#&!,$+]','',text)
#print(text.split(' '))